In [1]:
%cd ..

/home/gianscarpe/dev/event-camera/src


In [4]:
from experimenting.dataset import get_dataloader, DHPJointsDataset
from experimenting.dataset.indexes import get_dataset_params
from matplotlib import pyplot as plt
import numpy as np
from albumentations import Compose, CenterCrop
from albumentations.pytorch import ToTensor
import torch
from importlib import reload  
from experimenting.models import metrics
from experimenting.models import losses
from kornia import geometry
from omegaconf import DictConfig
import kornia
from experimenting.utils import get_file_paths
import numpy as np

%matplotlib inline

In [5]:
def display(img):          
        fig, ax = plt.subplots(ncols=img.shape[0], nrows=1,                              
                               figsize=(20,20))                                                                                   
        for i in range(img.shape[0]):                                                                
            ax[i].imshow(img[i])                                                  
            ax[i].axis('off')
        plt.show()

In [6]:
def plot_2d(dvs_frame, sample_gt, sample_pred):
    " To plot image and 2D ground truth and prediction "
    plt.figure()
    plt.imshow(dvs_frame, cmap='gray')
    plt.plot(sample_gt[:,1], sample_gt[:,0], '.', c='red', label='gt')
    plt.plot(sample_pred[:,1], sample_pred[:,0], '.', c='blue', label='pred')
    plt.legend()

In [7]:
# Augmentation
aug = Compose([ToTensor()])

In [8]:
data_dir = '/data/dhp19/time_count_dataset/movements_per_frame'
labels_dir = "/data/dhp19/time_count_dataset/labels_joints/"
test_subjects = [1, 2, 3, 4, 5]
cams = [2, 3]
max_w = 300
max_h = 342
batch_size = 16

hparams = DictConfig({'data_dir': data_dir, 'save_split':False, 'labels_dir' : labels_dir, 'test_subjects':test_subjects, 'cams': cams})
params = get_dataset_params(hparams)

.npy|.mat


In [9]:
d = DHPJointsDataset(file_paths = params['file_paths'], labels_dir = labels_dir, indexes=params['test_indexes'], max_w= max_w, max_h= max_h, transform=aug)

In [10]:
loader = get_dataloader(d, batch_size, 2)

In [13]:
batch = next(iter(loader))
x, y, m = batch

In [14]:
geometry.denormalize_pixel_coordinates(y[0], max_h, max_w)

tensor([[ 48.0000, 150.0000],
        [ 82.0000, 139.0000],
        [ 82.0000, 169.0000],
        [109.0000, 145.0000],
        [109.0000, 178.0000],
        [127.0000, 143.0000],
        [128.0000, 162.0000],
        [138.0000, 135.0000],
        [116.0000, 162.0000],
        [176.0000, 148.0000],
        [178.0000, 151.0000],
        [209.0000, 156.0000],
        [201.0000, 179.0000]])

In [15]:
import experimenting.models as models
import os
import experimenting.models.metrics as metrics
import experimenting.models.losses as losses
from experimenting.utils import get_joints_from_heatmap, average_loss

exp_name = 'exps_HourglassModel/exp_Adam_0.0003_no_aug_05-21-18-21/'
checkpoint_dir = f'/home/gianscarpe/dev/exps/timecount_pose/{exp_name}/checkpoints'
checkpoints = sorted(os.listdir(checkpoint_dir))
checkpoint_path = os.path.join(checkpoint_dir, checkpoints[0])
model = models.HourglassEstimator.load_from_checkpoint(checkpoint_path)
model = model.cpu()

In [16]:
mpjpe = metrics.MPJPE(reduction=average_loss)

In [17]:
with torch.no_grad():
    preds = model(x.cpu())[-1]

In [18]:
p_joints =  geometry.denormalize_pixel_coordinates(geometry.spatial_expectation2d(preds), d.max_h, d.max_w) 
gt_joints = geometry.denormalize_pixel_coordinates(y, max_h, max_w)

In [19]:
mpjpe(p_joints, gt_joints, m)

tensor(10.0457)

In [1]:
for i in range(batch_size):
    display(preds[i].numpy())
    plot_2d(x[i].squeeze(), p_joints[i], gt_joints[i])

NameError: name 'batch_size' is not defined